## 1. 데이터 불러오기

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

data_columns = train.columns.drop(['ID','y'])

print(f'train shape : {train.shape}')
print(f'test shape : {test.shape}')

train shape : (40118, 13)
test shape : (4986, 12)


## 2. 데이터 전처리

### 이상치 처리

In [46]:
# # IQR을 사용한 이상치 탐지 및 제거 함수 정의
# def remove_outliers(df, column):
#     # 1사분위수(25%)와 3사분위수(75%) 계산
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     # IQR 계산
#     IQR = Q3 - Q1
#     # 이상치 범위 설정 (1.5 * IQR보다 더 크거나 작은 값들)
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     # 이상치가 아닌 데이터만 반환
#     return df[(df[column] >= lower_bound)] 
#             #   & (df[column] <= upper_bound)]

# # target 컬럼에서 이상치 제거
# train = remove_outliers(train, 'y')
# print(f'train shape : {train.shape}')

In [47]:
from scipy import stats

def remove_outliers_zscore(df, column, threshold=3):
    """
    Z-score를 사용하여 이상치를 제거하는 함수

    Parameters:
    df (DataFrame): 데이터 프레임
    column (str): 이상치를 제거할 열 이름
    threshold (float): 이상치로 간주할 Z-score의 임계값 (기본값은 3)

    Returns:
    DataFrame: 이상치가 제거된 데이터 프레임
    """
    # Z-score 계산
    z_scores = stats.zscore(df[column])
    # Z-score가 threshold보다 작은 데이터만 반환
    return df[(z_scores >= -threshold) & (z_scores <= threshold)]

# target 컬럼에서 이상치 제거
train = remove_outliers_zscore(train, 'y')
print(f'train shape : {train.shape}')

train shape : (39264, 13)


### 로그 변환

In [48]:
from sklearn.preprocessing import PowerTransformer

# Yeo-Johnson 변환기 생성
pt = PowerTransformer(method='yeo-johnson')

# ID와 y를 제외한 나머지 컬럼에 대해 Yeo-Johnson 변환 수행
train_yeo_johnson = train.copy()
train_yeo_johnson[data_columns] = pt.fit_transform(train[data_columns])
test_yeo_johnson = test.copy()
test_yeo_johnson[data_columns] = pt.fit_transform(test[data_columns])

### 다중공선성

In [49]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

# 3. 다중공선성 처리 (VIF 계산)
X = train_yeo_johnson.drop(columns=['ID','x_8','y'])
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)
print(vif_data.VIF.sum())

  feature        VIF
0     x_0   2.780132
1     x_1  10.335470
2     x_2  10.485913
3     x_3   4.556827
4     x_4  10.355824
5     x_5  29.238399
6     x_6   7.363162
7     x_7  49.935635
8     x_9  62.782760
9    x_10  24.549525
212.3836482733996


### 시각화

In [50]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create histograms for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()
 
# for i, col in enumerate(train_yeo_johnson.columns[1:]):  # Skip the ID column
#     sns.histplot(train_yeo_johnson[col], ax=axes[i], kde=True)
#     axes[i].set_title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

# # Create box plots for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()

# for i, col in enumerate(train_yeo_johnson.columns[1:]):  # Skip the ID column
#     sns.boxplot(y=train_yeo_johnson[col], ax=axes[i])
#     axes[i].set_title(f'Box plot of {col}')

# plt.tight_layout()
# plt.show()

### 피쳐 생성

In [51]:
# train['x_8 * x_10'] = train['x_8'] * train['x_10']
# train['x_8 + x_10'] = train['x_8'] + train['x_10']
# train['x_8 - x_10'] = train['x_8'] - train['x_10']

# test['x_8 * x_10'] = test['x_8'] * test['x_10']
# test['x_8 + x_10'] = test['x_8'] + test['x_10']
# test['x_8 - x_10'] = test['x_8'] - test['x_10']

In [52]:
# train_x = train.drop(columns=['ID','x_2','y'])
# test_x = test.drop(columns=['ID','x_2'])
# train_y = train['y']

In [53]:
train_x = train_yeo_johnson.drop(columns=['ID','x_8','y'])
test_x = test_yeo_johnson.drop(columns=['ID','x_8'])
train_y = train_yeo_johnson['y']

### 데이터 스케일링

In [54]:
# # 3. 데이터 스케일링
# scaler = StandardScaler()
# # train_x 스케일링 후 데이터프레임으로 변환
# train_x_scaled = pd.DataFrame(scaler.fit_transform(train_x), columns=train_x.columns, index=train_x.index)
# # test_x 스케일링 후 데이터프레임으로 변환
# test_x_scaled = pd.DataFrame(scaler.transform(test_x), columns=test_x.columns, index=test_x.index)

## 3. 분석 모델 설계

In [55]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from optuna.integration import LightGBMPruningCallback

# 학습/테스트 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-6, 1.0, log=True),  # 축소된 범위
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-6, 1.0, log=True),  # 축소된 범위
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),  # 범위 축소
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),  # 범위 축소
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),  # 범위 축소
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50), 
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'force_col_wise': True  

    }

    # 모델 학습
    model = lgb.LGBMRegressor(**param)
    model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
              callbacks=[lgb.early_stopping(100), LightGBMPruningCallback(trial, "rmse")],
              )
    
    # 예측 및 평가
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    
    return rmse

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

trial = study.best_trial
best_params = trial.params

[I 2024-08-21 12:45:03,048] A new study created in memory with name: no-name-d526e488-0d6f-4651-9e12-dc67b682f594
C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-21 12:45:03,160] Trial 0 finished with value: 1.4664201757912418 and parameters: {'lambda_l1': 0.009486795791669061, 'lambda_l2': 0.0001932475699902813, 'num_leaves': 24, 'feature_fraction': 0.7578113430208191, 'bagging_fraction': 0.5378074393213446, 'bagging_freq': 1, 'min_child_samples': 45, 'learning_rate': 0.03754765027132902}. Best is trial 0 with value: 1.4664201757912418.


[LightGBM] [Warning] feature_fraction is set=0.7578113430208191, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7578113430208191
[LightGBM] [Warning] lambda_l1 is set=0.009486795791669061, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009486795791669061
[LightGBM] [Warning] lambda_l2 is set=0.0001932475699902813, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001932475699902813
[LightGBM] [Warning] bagging_fraction is set=0.5378074393213446, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5378074393213446
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.7578113430208191, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7578113430208191
[LightGBM] [Warning] lambda_l1 is set=0.009486795791669061, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009486795791669061
[LightGBM] [Warning] la

C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-21 12:45:03,315] Trial 1 finished with value: 1.4924955549127914 and parameters: {'lambda_l1': 5.065821007202573e-05, 'lambda_l2': 0.0002787609836896145, 'num_leaves': 47, 'feature_fraction': 0.857759425519746, 'bagging_fraction': 0.525341604225671, 'bagging_freq': 2, 'min_child_samples': 23, 'learning_rate': 0.018315624709063227}. Best is trial 0 with value: 1.4664201757912418.


Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 1.4925
[LightGBM] [Warning] feature_fraction is set=0.857759425519746, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.857759425519746
[LightGBM] [Warning] lambda_l1 is set=5.065821007202573e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.065821007202573e-05
[LightGBM] [Warning] lambda_l2 is set=0.0002787609836896145, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0002787609836896145
[LightGBM] [Warning] bagging_fraction is set=0.525341604225671, subsample=1.0 will be ignored. Current value: bagging_fraction=0.525341604225671
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8443656481982338, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8443656481982338
[LightGBM] [Warning] lambda_l1 is set=0.0037619403628038443, reg_alpha=0.0 will be ig

C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-21 12:45:03,480] Trial 2 finished with value: 1.4701579372883669 and parameters: {'lambda_l1': 0.0037619403628038443, 'lambda_l2': 3.4641098334637217e-06, 'num_leaves': 71, 'feature_fraction': 0.8443656481982338, 'bagging_fraction': 0.8189461714595461, 'bagging_freq': 2, 'min_child_samples': 19, 'learning_rate': 0.08378108430199374}. Best is trial 0 with value: 1.4664201757912418.
C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-21 12:45:03,586] Trial 3 finis

...

[LightGBM] [Warning] feature_fraction is set=0.5633982212111959, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5633982212111959
[LightGBM] [Warning] lambda_l1 is set=0.01915900076186344, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01915900076186344
[LightGBM] [Warning] lambda_l2 is set=2.724025574517497e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.724025574517497e-06
[LightGBM] [Warning] bagging_fraction is set=0.7450822804516959, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7450822804516959
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.5633982212111959, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5633982212111959
[LightGBM] [Warning] lambda_l1 is set=0.01915900076186344, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01915900076186344
[LightGBM] [Warning] lambda

[I 2024-08-21 12:45:14,601] Trial 95 pruned. Trial was pruned at iteration 0.
[I 2024-08-21 12:45:14,740] Trial 96 pruned. Trial was pruned at iteration 37.
[I 2024-08-21 12:45:14,798] Trial 97 pruned. Trial was pruned at iteration 0.


[LightGBM] [Warning] feature_fraction is set=0.5817294185159891, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5817294185159891
[LightGBM] [Warning] lambda_l1 is set=0.007957394371364697, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007957394371364697
[LightGBM] [Warning] lambda_l2 is set=3.859603395036652e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.859603395036652e-06
[LightGBM] [Warning] bagging_fraction is set=0.8019407116164557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8019407116164557
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.5817294185159891, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5817294185159891
[LightGBM] [Warning] lambda_l1 is set=0.007957394371364697, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007957394371364697
[LightGBM] [Warning] la

[I 2024-08-21 12:45:14,858] Trial 98 pruned. Trial was pruned at iteration 0.
[I 2024-08-21 12:45:14,981] Trial 99 pruned. Trial was pruned at iteration 46.


[LightGBM] [Warning] feature_fraction is set=0.5342364326754906, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5342364326754906
[LightGBM] [Warning] lambda_l1 is set=0.0003752016527660998, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0003752016527660998
[LightGBM] [Warning] lambda_l2 is set=2.4630213169989635e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4630213169989635e-05
[LightGBM] [Warning] bagging_fraction is set=0.765749945654162, subsample=1.0 will be ignored. Current value: bagging_fraction=0.765749945654162
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 31411, number of used features: 10
[LightGBM] [Warning] feature_fraction is set=0.5342364326754906, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5342364326754906
[LightGBM] [Warning] lambda_l1 

In [56]:
# 개별 모델 정의
lr = LinearRegression()
# rf = RandomForestRegressor(n_estimators=300,random_state=42)
lgbm = lgb.LGBMRegressor(**best_params)
# Voting Regressor로 앙상블 모델 생성
voting_regressor = VotingRegressor(estimators=[('lr', lr), ('lgbm', lgbm)],)

## 4. 모델 학습

In [57]:
voting_regressor.fit(train_x, train_y)

[LightGBM] [Warning] feature_fraction is set=0.5597449208536864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5597449208536864
[LightGBM] [Warning] lambda_l1 is set=0.0025226268392361142, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0025226268392361142
[LightGBM] [Warning] lambda_l2 is set=7.096017015048406e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.096017015048406e-06
[LightGBM] [Warning] bagging_fraction is set=0.7682455926758424, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7682455926758424
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.5597449208536864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5597449208536864
[LightGBM] [Warning] lambda_l1 is set=0.0025226268392361142, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0025226268392361142
[LightGBM] [Warning

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('lgbm',
                             LGBMRegressor(bagging_fraction=0.7682455926758424,
                                           bagging_freq=4,
                                           feature_fraction=0.5597449208536864,
                                           lambda_l1=0.0025226268392361142,
                                           lambda_l2=7.096017015048406e-06,
                                           learning_rate=0.08205836795754815,
                                           min_child_samples=32,
                                           num_leaves=21))])

In [58]:
# import matplotlib.pyplot as plt

# # 모델 학습 후 특성 중요도 추출
# importances = rf.feature_importances_
# indices = np.argsort(importances)[::-1]

# # 특성 중요도 시각화
# plt.figure(figsize=(10, 6))
# plt.title("Feature Importance")
# plt.bar(range(train_x_scaled.shape[1]), importances[indices], align="center")
# plt.xticks(range(train_x_scaled.shape[1]), train_x_scaled.columns[indices], rotation=90)
# plt.tight_layout()
# plt.show()

## 5. 예측값 생성
학습한 모델을 사용하여 예측값을 생성합니다.

In [59]:
y_pred = voting_regressor.predict(test_x)

[LightGBM] [Warning] feature_fraction is set=0.5597449208536864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5597449208536864
[LightGBM] [Warning] lambda_l1 is set=0.0025226268392361142, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0025226268392361142
[LightGBM] [Warning] lambda_l2 is set=7.096017015048406e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.096017015048406e-06
[LightGBM] [Warning] bagging_fraction is set=0.7682455926758424, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7682455926758424
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


## 5. 제출 파일 생성
submission 파일을 만들어서 제출합니다.

In [60]:
from datetime import datetime

submission = pd.read_csv('sample_submission.csv')

# 현재 날짜와 시간을 'YYYYMMDD_HHMMSS' 형식으로 포맷팅
today_datetime = datetime.today().strftime('%Y%m%d_%H%M')
submission['y'] = y_pred

# 포맷팅한 날짜와 시간으로 파일 이름 생성
submission.to_csv(f'./submission_{today_datetime}.csv', index=False)